In [ ]:
import json
from openai import OpenAI

client = OpenAI()

cv_text = """
Alex Johnson
Melbourne, Australia
Experience: 6 years
Skills: IFRS, variance analysis, month-end close, SAP, Excel

Experience:
Company: ABC Finance
Role: Financial Accountant
Dates: Jan 2020 – Present
- Reduced month-end close from 7 to 5 days
- Prepared IFRS-compliant reports
- Improved AR collection cycle by 25%

Education:
Bachelor of Commerce, Monash University, 2015
"""

schema = {
  "personal_info": {"name": "", "location": "", "years_experience": 0},
  "skills": [],
  "experience": [
    {"company": "", "title": "", "start_date": "", "end_date": "", "bullets": []}
  ],
  "education": [{"degree": "", "institution": "", "year": ""}]
}

prompt = f"""
Extract the following CV into this JSON schema: {json.dumps(schema, indent=2)}

Rules:
- Copy content exactly into the schema.
- Do not invent anything. Leave blank if missing.
- Only return valid JSON.

CV TEXT:
{cv_text}
"""

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[{"role": "system", "content": "You are a strict JSON extractor."},
              {"role": "user", "content": prompt}],
    response_format={"type": "json_object"}  # forces JSON mode if supported
)

state = json.loads(response.choices[0].message.content)

# Save state
with open("state.json", "w") as f:
    json.dump(state, f, indent=2)

print(json.dumps(state, indent=2))